# Aqueous speciation in the system Cu-Co-S-Cl-Na
In this notebook, I determine the speciation of Cu and Co-bearing complexes in a saline, S-bearing aqueous solution under hydrothermal conditions. 

## 1. Import Libraries

In [2]:
import numpy as np
import math 
import thermofun as fun
import reaktoro as rkt

## 2. Initialize a thermodynamic database; check the contents of the database

In [3]:
#database = fun.Database('./databases/ThermoFun_db/mines16-thermofun.json')
#database.appendData('./databases/Co_Cu_aq_speciation.json')
database = fun.Database('./databases/Co_Cu_aq_speciation.json')
#database = fun.Database('./databases/Willy_ThermoHub_db.json')

elements = database.mapElements()
print(elements.keys())
reactions = database.mapReactions()
print(reactions.keys())
substances = database.mapSubstances()
print(substances.keys())
engine = fun.ThermoEngine(database)


dict_keys(['Ag', 'Al', 'Ar', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Br', 'C', 'Ca', 'Cd', 'Ce', 'Cl', 'Co', 'Cr', 'Cs', 'Cu', 'Dy', 'Er', 'Eu', 'F', 'Fe', 'Ga', 'Gd', 'H', 'He', 'Hf', 'Hg', 'Ho', 'I', 'In', 'K', 'Kr', 'La', 'Li', 'Lu', 'Mg', 'Mn', 'Mo', 'N', 'Na', 'Nb', 'Nd', 'Ne', 'Ni', 'Nia', 'O', 'P', 'Pb', 'Pd', 'Pr', 'Pt', 'Ra', 'Rb', 'Re', 'Rh', 'Rn', 'Ru', 'S', 'Sb', 'Sc', 'Se', 'Si', 'Sm', 'Sn', 'Sr', 'Tb', 'Tc', 'Th', 'Ti', 'Tl', 'Tm', 'U', 'V', 'W', 'Xe', 'Y', 'Yb', 'Zn', 'Zr', 'Zz'])
dict_keys(['CoCl+', 'CoCl2@', 'CoCl3-', 'CoCl4-2', 'CoH2S+2', 'CoHS+'])
dict_keys(['Cl-', 'Co(s)', 'Co+++', 'Co+2', 'CoCl+', 'CoCl2@', 'CoCl3-', 'CoCl4-2', 'CoH2S+2', 'CoHS+', 'CoO', 'CoO2--', 'CoOH+', 'CoOH++', 'Cu(HS)2-', 'Cu(OH)2-', 'Cu+', 'Cu++', 'CuCl', 'CuCl+', 'CuCl2', 'CuCl2-', 'CuCl3-', 'CuCl3--', 'CuCl4--', 'CuHS', 'CuO', 'CuO2--', 'CuOH', 'CuOH+', 'H+', 'H2O@', 'H2S@', 'HCl@', 'HCuO2-', 'HS-', 'HSO4-', 'Na+', 'NaCl@', 'NaOH@', 'OH-', 'SO4--'])


## 3. Set temperature and pressure conditions

In [4]:
T = 200 + 273.15 # [K]
P = rkt.waterSaturatedPressureWagnerPruss(T).val/100000 # [bar] saturated water vapor pressure

## 4. Set pH and chlorinity conditions

In [5]:
# Target pH 
pH = 3
# Target chlorinity
c_Cl = 1 # [mol*kg^-1]

## 5. Define and construct the chemical system

In [21]:
# Define the chemical system
editor = rkt.ChemicalEditor(database)
# editor.addAqueousPhaseWithElements([
# 'H', 'O', 'Cl', 'Na', 'S', 'Co'])
# editor.addAqueousPhase(
#     ['Cl-', 'Co+++', 'Co+2', 'CoCl+', 'CoCl2@', 
#      'CoCl3-', 'CoCl4-2', 'CoH2S+2', 'CoHS+', 'CoO', 'CoO2--', 
#      'CoOH+', 'CoOH++', 'Cu(HS)2-', 'Cu(OH)2-', 'Cu+', 'Cu++', 
#      'CuCl', 'CuCl+', 'CuCl2', 'CuCl2-', 'CuCl3-', 'CuCl3--', 
#      'CuCl4--', 'CuHS', 'CuO', 'CuO2--', 'CuOH', 'CuOH+', 'H+', 
#      'H2O@', 'H2S@', 'HCl@', 'HCuO2-', 'HS-', 'HSO4-', 'Na+', 
#      'NaCl@', 'NaOH@', 'OH-', 'SO4--'])
# editor.addAqueousPhase(
#     ['Cl-', 'Cu+', 'Cu++', 'CuCl', 'CuCl+', 'CuCl2', 
#      'CuCl2-', 'CuCl3-', 'CuCl3--', 'CuCl4--', 'H+', 
#      'H2O@', 'HCl@', 'Na+', 'NaCl@', 'NaOH@', 'OH-'])
editor.addAqueousPhase(
    ['Cl-', 'Cu+', 'Cu++', 'CuCl2', 'CuCl2-', 
     'H2O@'])
# Construct the chemical system
system = rkt.ChemicalSystem(editor)

## 6. Define the chemical equilibrium problem

In [22]:
problem = rkt.EquilibriumProblem(system)
#problem.add("Co",0.03,"mol")
problem.add("Cu",0.015,"mol")
problem.add("H2O",1,"kg")
problem.add("HCl",10**-pH,"mol")
problem.add("Cl",c_Cl - 10**-pH,"mol")
problem.setPressure(P, "bar")
problem.setTemperature(T, "kelvin")

## 7. Calculate the chemical equilibrium state

In [23]:
state = rkt.equilibrate(problem)
print(state)

Temperature [K]          Temperature [C]          Pressure [Pa]            Pressure [bar]           
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
473.15                   200                      1.55494e+06              15.5494                  
Element                  Amount [mol]             Aqueous [mol]            Dual Potential [kJ/mol]  
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Cl                       1                        1                        38.2109                  
Cu                       0.985                    0.985                    -351.972                 
H                        111.018           

In [ ]:
# T = 298.15 # [K]
# P = 1e5 # [Pa]
# dGr = rkt.engine.thermoPropertiesReaction(T, P, "CoCl4-2").reaction_gibbs_energy # [J*mol^-1]
# print(dGr.val) 